DIJKSTRA

In [17]:
import numpy as np
import random

In [18]:
def plus_court(dist , visités):
    #initialisation
    min_dist=np.inf
    min_indice=-1

    for i in range(len(dist)):
        if not visités[i] and dist[i]<min_dist:
            min_dist=dist[i]
            min_indice=i

    return min_indice

In [19]:
def Dijkstra(adj , s):

    n=len(adj)
    dist=[np.inf]*n
    père=[None]*n
    visités=[False]*n

    dist[s]=0
    père[s]=-1

    while True :
        u=plus_court(dist,visités)
        if u==-1:
            break
        visités[u]=True

        for v in range(n):
            if not visités[v] and adj[u][v]!=0 and dist[u] + adj[u][v]<dist[v]:
                dist[v]=dist[u]+adj[u][v]
                père[v]=u

                
    return dist , père

In [63]:
adj_matrix = [
    [0, 2, 0, 3, 0, 0, 0, 0],
    [2, 0, 6, 0, 0, 3, 3, 0],
    [0, 6, 0, 0, 1, 1, 0, 0],
    [3, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 1, 0, 0, 0, 1, 0],
    [0, 3, 1, 0, 0, 0, 0, 4],
    [0, 3, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 6, 0, 4, 1, 0]
]

s= 3
distances, parents = Dijkstra(adj_matrix, s)
print(distances,parents)

[3, 5, 9, 0, 8, 8, 7, 6] [3, 0, 4, -1, 6, 1, 7, 3]


In [61]:
def get_shortest_paths(parents):
    shortest_paths=[]
    for i in range(0, len(parents)):
        shortest_path_to_i=[i]
        while parents[i]!=-1:
            shortest_path_to_i.append(parents[i])
            i=parents[i]
        shortest_path_to_i.reverse()
        shortest_paths.append(shortest_path_to_i)
    return shortest_paths

In [64]:
get_shortest_paths(parents)

[[3, 0],
 [3, 0, 1],
 [3, 7, 6, 4, 2],
 [3],
 [3, 7, 6, 4],
 [3, 0, 1, 5],
 [3, 7, 6],
 [3, 7]]

![dijkstra_animation](dijkstra_animation.png)

DIJKSTRA dynamique

let us try to implement the dijkstra algorithm in a dynamic graph case 
we have seen that it's only possible in the case of 'un graphe dynamique par intervalles FIFO' , where the problem is not 'NP difficile'
first of all , the input will be 'un graphe déterministe statique FIFO' , that will be transformed to 'un graphe dynamique par intervalles FIFO'

once this inisialisation is done , we'll calculate the weight(pondération) according to the probability law that it follows 
with the ponderation known , we can now execute de classic Dijkstra , and we'll have our minimum cost path 


we have to mention that a fifo static graph , when transformed , will give a fifo dynamic graph


In [23]:
def transformateur_to_dynamic_graph(G):
    dynamic_ponderations=[]
    for i in range(len(G)):
        row=[]
        for j in range(len(G[0])):
            if G[i][j]!=0:
                row.append([G[i][j]-0.5,G[i][j]+0.5])#assert that the given graph weights are all greater than 1
            else:
                row.append(0)
        dynamic_ponderations.append(row)

    #dynamic_ponderations = np.array(dynamic_ponderations)
    #dynamic_ponderations.reshape(len(G),len(G[0]),2)

    return dynamic_ponderations

In [24]:
G = [
    [0, 2, 0, 3, 0, 0, 0, 0],
    [2, 0, 6, 0, 0, 3, 3, 0],
    [0, 6, 0, 0, 1, 1, 0, 0],
    [3, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 1, 0, 0, 0, 1, 0],
    [0, 3, 1, 0, 0, 0, 0, 4],
    [0, 3, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 6, 0, 4, 1, 0]
]
transformateur_to_dynamic_graph(G)

[[0, [1.5, 2.5], 0, [2.5, 3.5], 0, 0, 0, 0],
 [[1.5, 2.5], 0, [5.5, 6.5], 0, 0, [2.5, 3.5], [2.5, 3.5], 0],
 [0, [5.5, 6.5], 0, 0, [0.5, 1.5], [0.5, 1.5], 0, 0],
 [[2.5, 3.5], 0, 0, 0, 0, 0, 0, [5.5, 6.5]],
 [0, 0, [0.5, 1.5], 0, 0, 0, [0.5, 1.5], 0],
 [0, [2.5, 3.5], [0.5, 1.5], 0, 0, 0, 0, [3.5, 4.5]],
 [0, [2.5, 3.5], 0, 0, [0.5, 1.5], 0, 0, [0.5, 1.5]],
 [0, 0, 0, [5.5, 6.5], 0, [3.5, 4.5], [0.5, 1.5], 0]]

In [50]:
def dijkstra_dynamic(G, s, alfa, nb_iterations=100):
    G_dynamic = transformateur_to_dynamic_graph(G)
    num_nodes = len(G_dynamic)
    results = []

    for _ in range(nb_iterations):
        pondérations_dinamiques = np.zeros((num_nodes, num_nodes))
        
        
        for j in range(num_nodes):
            for k in range(len(G_dynamic[j])):
                
                if G_dynamic[j][k] != 0:
                    fluctuation = random.uniform(-0.9, 0.9)
                    G_dynamic[j][k][0] += fluctuation
                    G_dynamic[j][k][1] -= fluctuation
                    pondérations_dinamiques[j][k] = G_dynamic[j][k][0] + (G_dynamic[j][k][1] - G_dynamic[j][k][0]) * alfa  # loi équiprobable
        #print('pondérations_dinamiques',pondérations_dinamiques)
        dist, parents = Dijkstra(pondérations_dinamiques, s)
        paths = get_shortest_paths(parents)
        results.append(paths)
    
    return results

# Example graph (weighted adjacency matrix)
G = [
    [0, 2, 0, 3, 0, 0, 0, 0],
    [2, 0, 6, 0, 0, 3, 3, 0],
    [0, 6, 0, 0, 1, 1, 0, 0],
    [3, 0, 0, 0, 0, 0, 0, 6],
    [0, 0, 1, 0, 0, 0, 1, 0],
    [0, 3, 1, 0, 0, 0, 0, 4],
    [0, 3, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 6, 0, 4, 1, 0]
]
s = 0
alfa = 0.6
results = dijkstra_dynamic(G, s, alfa)
for iteration, paths in enumerate(results):
    if results[iteration]!=results[iteration-1]:
        print(f"t{iteration}: {paths}")


t0: [[0, 1], [0, 1, 5, 2], [0, 3], [0, 1, 6, 4], [0, 1, 5], [0, 1, 6], [0, 1, 6, 7]]
t44: [[0, 1], [0, 1, 6, 4, 2], [0, 3], [0, 1, 6, 4], [0, 1, 5], [0, 1, 6], [0, 1, 6, 7]]
t45: [[0, 1], [0, 1, 5, 2], [0, 3], [0, 1, 6, 4], [0, 1, 5], [0, 1, 6], [0, 1, 6, 7]]
t51: [[0, 1], [0, 1, 5, 2], [0, 3], [0, 1, 6, 4], [0, 1, 5], [0, 1, 6], [0, 1, 5, 7]]
t55: [[0, 1], [0, 1, 5, 2], [0, 3], [0, 1, 5, 2, 4], [0, 1, 5], [0, 1, 6], [0, 1, 5, 7]]
t57: [[0, 1], [0, 1, 5, 2], [0, 3], [0, 1, 6, 4], [0, 1, 5], [0, 1, 6], [0, 1, 5, 7]]
t58: [[0, 1], [0, 1, 5, 2], [0, 3], [0, 1, 5, 2, 4], [0, 1, 5], [0, 1, 6], [0, 1, 5, 7]]
t88: [[0, 1], [0, 1, 5, 2], [0, 3], [0, 1, 5, 2, 4], [0, 1, 5], [0, 1, 5, 2, 4, 6], [0, 1, 5, 7]]
t90: [[0, 1], [0, 1, 5, 2], [0, 3], [0, 1, 5, 2, 4], [0, 1, 5], [0, 1, 6], [0, 1, 5, 7]]
